## Import libraries

In [1]:
import pandas as pd
from datetime import date

## Exploring given a dataset with real data ftom the main Portuguese football league

In [2]:
pt_league_attendance = pd.read_csv('./pt_league_attendance_train.csv',sep = ';')

pt_league_attendance = pt_league_attendance.dropna()

In [3]:
pt_league_attendance.head()

,Competition,Date,Round,Host,Visitor,GoalsHHT,GoalsVHT,ResultHT,GoalsHFT,GoalsVFT,ResultFT,Attendance
0,Liga Sagres,14/08/09,Jornada 1,Leixoes SC,Os Belenenses,0.0,0.0,D,0.0,0.0,D,3034.0
1,Liga Sagres,15/08/09,Jornada 1,CD Nacional,Sporting CP,1.0,0.0,H,1.0,1.0,D,3877.0
2,Liga Sagres,15/08/09,Jornada 1,SC Braga,A. Academica,0.0,0.0,D,1.0,0.0,H,9402.0
3,Liga Sagres,16/08/09,Jornada 1,FC P.Ferreira,FC Porto,1.0,0.0,H,1.0,1.0,D,4654.0
4,Liga Sagres,16/08/09,Jornada 1,Leiria,Rio Ave FC,0.0,1.0,A,1.0,1.0,D,1808.0


## Custom dataset with Stadium name and capacity of hosts stadiums

In [4]:
pt_stadium_information = pd.read_csv('./portugal_stadium_db.csv', sep = ',')

In [5]:
pt_stadium_information.head()

,Host,Capacity,City,Stadium
0,Leixoes SC,12.035,Matosinhos,Estádio do Mar
1,CD Nacional,5.132,Funchal,Estádio do Madeira
2,SC Braga,30.286,Braga,Estádio Municipal de Braga
3,FC P.Ferreira,9.077,Pacos de Ferreira,Estádio de Mata Real
4,Leiria,23.758,Leiria,Estádio Municipal de Leiria


## Joining datasets

In [6]:
pt_league_attendance = pd.merge(pt_league_attendance,\
                                      pt_stadium_information,\
                                      on='Host')

In [7]:
pt_league_attendance.head()

,Competition,Date,Round,Host,Visitor,GoalsHHT,GoalsVHT,ResultHT,GoalsHFT,GoalsVFT,ResultFT,Attendance,Capacity,City,Stadium
0,Liga Sagres,14/08/09,Jornada 1,Leixoes SC,Os Belenenses,0.0,0.0,D,0.0,0.0,D,3034.0,12.035,Matosinhos,Estádio do Mar
1,Liga Sagres,30/08/09,Jornada 3,Leixoes SC,Rio Ave FC,0.0,0.0,D,0.0,0.0,D,3629.0,12.035,Matosinhos,Estádio do Mar
2,Liga Sagres,18/09/09,Jornada 5,Leixoes SC,Vitoria SC,1.0,1.0,D,3.0,1.0,H,3560.0,12.035,Matosinhos,Estádio do Mar
3,Liga Sagres,04/10/09,Jornada 7,Leixoes SC,Leiria,1.0,1.0,D,3.0,2.0,H,3193.0,12.035,Matosinhos,Estádio do Mar
4,Liga Sagres,08/11/09,Jornada 10,Leixoes SC,CD Nacional,1.0,2.0,A,2.0,4.0,A,3257.0,12.035,Matosinhos,Estádio do Mar


## Transforming not numeric data to numerics one

In [8]:
#Competition column
competition_mapping = {
                       'Liga Sagres': 1.0,\
                       'Liga Zon Sagres': 2.0,\
                       'Liga NOS': 3.0
                      }

#Results columns
results_mapping = {
                   'D': 0.0,\
                   'H': 1.0,\
                   'A': 2.0
                  }

#Teams columns
teams_db = pd.concat(\
                     [pt_league_attendance['Visitor']\
                      ,pt_league_attendance['Host']],\
                     axis=0)\
                    .drop_duplicates()\
                    .reset_index(drop=True)

teams_mapping = teams_db.to_dict()

teams_mapping = {v: float(k) for k, v in teams_mapping.items()}


pt_league_attendance = pt_league_attendance.replace(\
                                                    {'Host': teams_mapping,\
                                                     'Visitor': teams_mapping,\
                                                     'ResultHT': results_mapping,\
                                                     'ResultFT': results_mapping,\
                                                     'Competition': competition_mapping 
                                                    })


#Transforming Round column applying a function.

pt_league_attendance['Round'] = pt_league_attendance['Round']\
                                        .apply(lambda x: float(x.split(' ')[1]))

#Transforming Date column applying a function
#the function obtains the number of days has passed of the year that the match was.
def number_of_days_of_date(x):
    day_month_year = list(map(int, x.split('/')))
    current_date = date(day_month_year[2], day_month_year[1], day_month_year[0])
    initial_date = date(day_month_year[2], 1, 1)
    return float((current_date - initial_date).days)

#this function transform the string date into a date object for future comparisons
def to_date(x):
    day_month_year = list(map(int, x.split('/')))
    return date(day_month_year[2], day_month_year[1], day_month_year[0])

pt_league_attendance['DayOfTheYear'] = pt_league_attendance['Date']\
                                        .apply(number_of_days_of_date)

pt_league_attendance['Date'] = pt_league_attendance['Date']\
                                        .apply(to_date)

pt_league_attendance = pt_league_attendance\
                            .rename(index=str, columns={"Date": "DayOfTheYear"})




In [9]:
pt_league_attendance.head()

,Competition,DayOfTheYear,Round,Host,Visitor,GoalsHHT,GoalsVHT,ResultHT,GoalsHFT,GoalsVFT,ResultFT,Attendance,Capacity,City,Stadium,DayOfTheYear
0,1.0,0009-08-14,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3034.0,12.035,Matosinhos,Estádio do Mar,225.0
1,1.0,0009-08-30,3.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3629.0,12.035,Matosinhos,Estádio do Mar,241.0
2,1.0,0009-09-18,5.0,15.0,2.0,1.0,1.0,0.0,3.0,1.0,1.0,3560.0,12.035,Matosinhos,Estádio do Mar,260.0
3,1.0,0009-10-04,7.0,15.0,3.0,1.0,1.0,0.0,3.0,2.0,1.0,3193.0,12.035,Matosinhos,Estádio do Mar,276.0
4,1.0,0009-11-08,10.0,15.0,4.0,1.0,2.0,2.0,2.0,4.0,2.0,3257.0,12.035,Matosinhos,Estádio do Mar,311.0


## Adding new column with the performance of the team